In [1]:
from bs4 import BeautifulSoup
import os

import re

In [6]:
from polyglot.text import Text

In [ ]:
from termcolor import colored

In [268]:
from dateparser.calendars.jalali import JalaliCalendar

In [459]:
from functools import reduce

In [434]:
from datetime import datetime

In [210]:
soups = dict()
SAMPLES_DIR = 'test-files/'
for filename in os.listdir(SAMPLES_DIR):
    if filename.startswith('sample_'):
        with open(
            '{dir}/{filename}'.format(dir=SAMPLES_DIR, filename=filename),
            'r'
        ) as f:
            soups[filename[:-5]] = BeautifulSoup(f.read(), 'html5lib')

todo

- parse dates
- (name, national id number(?), position(?))
- type of company?
- learn/compile a dictionary?

In [777]:
class Advertisement:
    
    PERSIAN = 'fa'
    FA_NUMERALS = [chr(i) for i in range(ord(u'\u06F0'), ord(u'\u06F0') + 10)]
    FA_TO_HA = {fa_numeral: str(i) for i, fa_numeral in enumerate([chr(i) for i in range(ord(u'\u06F0'), ord(u'\u06F0') + 10)])}
    
    DICTIONARY = {
        'national_id_tag': {
            'national_id': 'شناسه\sملی',
            'national_number': 'شماره\sملی',
        },
        'registration_number_tag': {
            'registration_number': 'شماره\sثبت'
        }
    }
    
    SELECTORS = {
        'journal_tracking_number': {'id': 'cphMain_lblReferenceNumber'},
        'newspaper_date': {'id': 'cphMain_lblNewsPaperDate'},
        'newspaper_no': {'id': 'cphMain_lblNewspaperNo'},
        'newspaper_type': {'id': re.compile(r'cphMain_lblNewsPaper.*Type')},
        'page_number': {'id': 'cphMain_lblPageNumber'},
        'registration_date': {'id': 'cphMain_lblNewsDate'},
        'registration_letter_number': {'id': 'cphMain_lblIndikatorNumber'},
        'title': {'id': 'cphMain_lblNewsTitle'},
        'text': {'class_': 'Jus'},
    }
    
    STYLES = {
        'company_name': lambda x: x,
        'company_type': lambda x: x,
        'registration_number_tag': lambda x: colored(x, color='red'),
        'registration_number': lambda x: colored(x, color='white', on_color='on_red'),
        'national_id_tag': lambda x: colored(x, color='green'),
        'national_id': lambda x: colored(x, color='white', on_color='on_green'),
        'person': lambda x: colored(x, color='white', on_color='on_blue'),
        'date': lambda x: colored(x, color='white', on_color='on_magenta'),
        'number': lambda x: colored(x, on_color='on_white')
    }
    
    def __init__(self, soup):

        self.newspaper_number = {'fa': getattr(soup.find(**self.SELECTORS['newspaper_no']), 'text', None)}
        self.newspaper_number['ha'] = self.fa_number_to_ha(self.newspaper_number['fa'])

        self.newspaper_type = getattr(soup.find(**self.SELECTORS['newspaper_type']), 'text', None)

        self.page_number_fa = getattr(soup.find(**self.SELECTORS['page_number']), 'text', None)
        self.page_number_ha = self.fa_number_to_ha(self.page_number_fa)


        self.date_fa = getattr(soup.find(**self.SELECTORS['newspaper_date']), 'text', None)
        self.date_ha = self.fa_number_to_ha(self.date_fa)
        self.date_dt = self.fa_date_to_dt(self.date_fa)

        self.registration_letter_number_fa = getattr(soup.find(**self.SELECTORS['registration_letter_number']), 'text', None)
        self.registration_letter_number_ha = self.fa_number_to_ha(self.registration_letter_number_fa)

        self.registration_date_fa = getattr(soup.find(**self.SELECTORS['registration_date']), 'text', None)
        self.registration_date_ha = self.fa_number_to_ha(self.registration_date_fa)
        self.registration_date_dt = self.fa_date_to_dt(self.registration_date_fa)

        self.journal_tracking_number_fa = getattr(soup.find(**self.SELECTORS['journal_tracking_number']), 'text', None)
        self.journal_tracking_number_ha = self.fa_number_to_ha(self.journal_tracking_number_fa)

        self.title = None
        self.title_fa = soup.find(**self.SELECTORS['title'])
        if self.title_fa is not None:
            self.title_fa = self.title_fa.text.strip()
            self.title = Text(self.title_fa, hint_language_code=self.PERSIAN)
            self.title_sequence = self.title.word_tokenizer.transform(Sequence(self.title.raw))
            self.title_split = [self.title.raw[start:end] for start, end in zip(self.title_sequence.idx, self.title_sequence.idx[1:])]

        self.text = None
        self.text_fa = soup.find(**self.SELECTORS['text'])
        if self.text_fa is not None:
            self.text_fa = self.text_fa.text.strip()
            self.text = Text(self.text_fa, hint_language_code=self.PERSIAN)
            self.text_sequence = self.text.word_tokenizer.transform(Sequence(self.text.raw))
            self.text_split = [self.text.raw[start:end] for start, end in zip(self.text_sequence.idx, self.text_sequence.idx[1:])]

        self.parsed = {'title': dict(), 'text': dict()}

        self.parsed['title']['entities'] = getattr(self.title, 'entities', None)
        self.parsed['text']['entities'] = getattr(self.text, 'entities', None)

        self.parsed['title']['numbers'] = self.get_numbers(self.title_fa)
        self.parsed['text']['numbers'] = self.get_numbers(self.text_fa)

        self.parsed['title']['dates'] = self.get_dates(self.parsed['title']['numbers'])
        self.parsed['text']['dates'] = self.get_dates(self.parsed['text']['numbers'])

        self.parsed['title']['tags'] = self.get_tags(self.title_fa)
        self.parsed['text']['tags'] = self.get_tags(self.text_fa)
    
    @staticmethod
    def fa_number_to_ha(number):
        if number is None:
            return None
        ha = ''.join([Advertisement.FA_TO_HA.get(c, c) for c in number])
        try:
            return int(ha)
        except ValueError:
            return ha
        
    @staticmethod
    def fa_date_to_dt(date):
        if len(date) < 8:
            return None
        return (JalaliCalendar(date).get_date() or dict()).get('date_obj')
    
    @staticmethod
    def get_numbers(text):
        return list(re.finditer(r'([\u06F0-\u06F9]+[:/-]?)+', text or ''))
    
    @staticmethod
    def get_dates(numbers):
        return [Advertisement.fa_date_to_dt(m.group()) for m in (numbers or [])]
    
    @staticmethod
    def get_tags(text):
        return {
            tag: list(re.finditer(fa, text or ''))
            for tag, matches in Advertisement.DICTIONARY.items()
            for en, fa in matches.items()
        }
    
    def stylize(self, title_or_text):
        text = getattr(self, title_or_text, None)
        words = getattr(self, title_or_text + '_split', None)
        if text is None or words is None:
            return
        chars = text.raw
        words = words.copy()

        char_annotations = sorted([
            (number_match.span(), 'number')
            if self.parsed[title_or_text]['dates'][i] is None else
            (number_match.span(), 'date')
            for i, number_match in enumerate(self.parsed[title_or_text]['numbers'])
        ] + [
            (tag_match.span(), tag_name)
            for tag_name, tag_matches
            in self.parsed[title_or_text]['tags'].items()
            for tag_match in tag_matches
        ], key=lambda x: x[0][0])
        word_annotations = sorted([
            (entity.start, entity.end)
            for entity in self.parsed[title_or_text]['entities']
            if entity.tag == 'I-PER'
        ], key=lambda x: x[0])

        c = 0
        w = 0
        print_buffer = []
        while len(words) > 0:
            if len(char_annotations) > 0 and c == char_annotations[0][0][0]:
                # annotate stuff from char
                next_annotation = char_annotations.pop(0)
                annotation_length = next_annotation[0][1] - next_annotation[0][0]
                annotation_type = next_annotation[1]
                annotation = ''
                while len(annotation) < annotation_length:
                    next_word = words.pop(0)
                    annotation += next_word
                    c += len(next_word)
                    w += 1 if not re.match(r'\s+', next_word) else 0
                print_buffer.append(self.STYLES[annotation_type](annotation))
            elif (
                len(word_annotations) > 0 and
                w == word_annotations[0][0] and
                not re.match(r'\s+', words[0])
            ):
                # annotate person name
                next_annotation = word_annotations.pop(0)
                annotation_end = next_annotation[1]
                annotation = ''
                while w < annotation_end:
                    next_word = words.pop(0)
                    annotation += next_word
                    c += len(next_word)
                    w += 1 if not re.match(r'\s+', next_word) else 0
                print_buffer.append(self.STYLES['person'](annotation))
            else:
                # print normal stuff
                next_word = words.pop(0)
                print_buffer.append(next_word)
                c += len(next_word)
                w += 1 if not re.match(r'\s+', next_word) else 0
            
        print(''.join(print_buffer))

In [778]:
a = Advertisement(soups['sample_1'])

In [779]:
a.__dict__

{'newspaper_number': {'fa': '۲۰۶۳۲', 'ha': 20632},
 'newspaper_type': 'شهرستان',
 'page_number_fa': '۳۱',
 'page_number_ha': 31,
 'date_fa': '۱۳۹۴/۱۰/۱۴',
 'date_ha': '1394/10/14',
 'date_dt': datetime.datetime(2016, 1, 4, 0, 0),
 'registration_letter_number_fa': '۱۳۹۴۳۰۴۱۱۱۳۸۰۰۰۲۳۳',
 'registration_letter_number_ha': 139430411138000233,
 'registration_date_fa': '۱۳۹۴/۵/۲۷',
 'registration_date_ha': '1394/5/27',
 'registration_date_dt': datetime.datetime(2015, 8, 18, 0, 0),
 'journal_tracking_number_fa': '۹۴۰۵۲۹۷۶۳۹۹۱۱۹۷',
 'journal_tracking_number_ha': 940529763991197,
 'title': Text("آگهی تغییرات شرکت هخامنش باستان پارس شركت سهامي خاص به شماره ثبت ۱۴۳۳ و شناسه ملی ۱۰۵۳۰۳۳۷۹۲۵"),
 'title_fa': 'آگهی تغییرات شرکت هخامنش باستان پارس شركت سهامي خاص به شماره ثبت ۱۴۳۳ و شناسه ملی ۱۰۵۳۰۳۳۷۹۲۵',
 'title_sequence': <polyglot.base.Sequence at 0x11aeee940>,
 'title_split': ['آگهی',
  ' ',
  'تغییرات',
  ' ',
  'شرکت',
  ' ',
  'هخامنش',
  ' ',
  'باستان',
  ' ',
  'پارس',
  ' ',
  'شركت',
  ' ',

In [780]:
a.stylize('title')

آگهی تغییرات شرکت هخامنش باستان پارس شركت سهامي خاص به شماره ثبت ۱۴۳۳ و شناسه ملی ۱۰۵۳۰۳۳۷۹۲۵


In [781]:
a.stylize('text')

به استناد صورتجلسه هیئت مدیره مورخ ۳۱/۰۳/۱۳۹۴ تصمیمات ذیل اتخاذ شد: ـ سمت اعضاء هیئت مدیره به قرار ذیل تعیین گردیدند: خانم مریم باستان با شماره ملی ۲۲۹۷۶۸۲۴۶۸به سمت رئیس هیئت مدیره ـ خانم مریم فتاحیبا شماره ملی ۰۰۶۹۴۴۷۱۶۰ به سمت نائب رئیس هیئت مدیره ـ خانم مرجان باستان با شماره ملی ۲۲۹۶۹۷۶۳۷۹ به سمت  مدیرعامل و عضو هیئت مدیره انتخاب گردیدند. ـ دارندگان حق امضا: کلیه اوراق و اسناد رسمی و بهادار و بانکی و تعهدآور و قراردادها ونامه های عادی و اداری به امضا خانم مرجان باستان ( مدیرعامل) بتنهائیهمراه با مهر شرکت معتبر می باشد.
ش۹۴۰۵۲۹۷۶۳۹۹۱۱۹۷  اداره ثبت شرکت ها و موسسات غیرتجاری مرودشت
